# Distance Classes to Nearest Soil Samples

Alessandro Samuel-Rosa, Taciara Zborowski Horst

2025-07-07

This script calculates the distance from each pixel of a reference map to the nearest soil sample point used for particle size distribution modeling across the Brazilian biomes. It is designed to produce a map that visualizes the spatial representativeness of this specific sampling network.

The core calculation uses the efficient `ee.Image.fastDistanceTransform` method. The resulting distance-in-pixels is converted to a precise distance-in-meters using `ee.Image.pixelArea().sqrt()`, which accounts for map projection distortions.

Finally, the continuous distance values are reclassified into six distinct classes (e.g., 0-50 m, 50-250 m, up to >10,000 m). The output is a single-band, 8-bit integer image where each pixel value from 1 to 6 corresponds to one of these distance classes.

License: MIT

In [11]:
# --- 1. IMPORT LIBRARIES & INITIALIZE ---
import ee
import geemap

# Authenticate and initialize the Earth Engine project.
# ee.Authenticate() # Run once in your environment if needed.
ee.Initialize(project='mapbiomas-solos-workspace')

In [12]:
# --- 2. DEFINE ASSETS ---
# Soil sampling points.
samplingPoints = ee.FeatureCollection('projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm')

# Reference soil property map.
# CORRECTED LINE: Select a valid band name from your image.
soilPropertyMap = ee.Image('projects/mapbiomas-workspace/SOLOS/PRODUTOS_C02/c02v2/mapbiomas_soil_collection2_v2_clay').select('clay_000_010cm')

# Area of Interest (AOI).
aoi = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/biomas_IBGE_250mil')

In [13]:
# --- 3. PROCESS DATA: CALCULATE DISTANCE CLASSES ---
# Create a base grid from the soil map for perfect alignment.
sourceImage = soilPropertyMap.multiply(0).paint(samplingPoints, 1)

# Apply the Fast Distance Transform (FDT).
fdt = sourceImage.fastDistanceTransform(512, 'pixels', 'squared_euclidean')

# Convert to a precise distance in meters using pixelArea().
distanceInPixels = fdt.select('distance').sqrt()
pixelSize = ee.Image.pixelArea().sqrt()
distanceInMeters = distanceInPixels.multiply(pixelSize)

# Reclassify the continuous distance into 6 discrete, meaningful classes.
distanceClasses = ee.Image(6) \
  .where(distanceInMeters.lte(10000), 5) \
  .where(distanceInMeters.lte(5000), 4) \
  .where(distanceInMeters.lte(1000), 3) \
  .where(distanceInMeters.lte(250), 2) \
  .where(distanceInMeters.lte(50), 1) \
  .toUint8()

# Clip the final raster to the boundary of the AOI.
finalClasses = distanceClasses.clip(aoi)

In [14]:
# --- 4. VISUALIZE THE RESULT ---
# Create an interactive map object.
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Define visualization parameters for the 6 classes.
visParams = {
  'min': 1,
  'max': 6,
  'palette': ['#0000FF', '#00A0FF', '#00FF00', '#FFFF00', '#FFA500', '#FF0000']
}

# Add the final layer to the map and center the view.
Map.addLayer(finalClasses, visParams, 'Geostatistical Distance Classes')
Map.centerObject(aoi, 4)

# Display the interactive map in your notebook.
Map


Map(center=[-10.62046648393736, -53.18363392405513], controls=(WidgetControl(options=['position', 'transparent…

In [15]:
# --- 4. Prepare Image for Distance Calculation (No Reprojection) ---
print("\nRasterizing points (using default projection/scale)...")

# Reduce to Image - uses GEE default projection/scale
point_presence_image = soil_points_with_constant.reduceToImage(
    properties=['constant'],
    reducer=ee.Reducer.sum().unweighted()
).unmask(0)

# Create the source image (1 where points are, 0 elsewhere)
source_image = point_presence_image.gt(0)


Rasterizing points (using default projection/scale)...


NameError: name 'soil_points_with_constant' is not defined

In [ ]:
# --- 5. Calculate Distance in Pixels ---
print("Calculating distance in PIXELS using fastDistanceTransform...")
distance_image_multiband = source_image.fastDistanceTransform(
    neighborhood=NEIGHBORHOOD_PIXELS,
    metric='squared_euclidean'
)
distance_pixels_image = distance_image_multiband.select('distance').sqrt()

Calculating distance in PIXELS using fastDistanceTransform...


In [ ]:
# --- 6. Convert to Approximate Meters ---
print("Converting pixel distance to approximate METERS using pixelArea()...")
pixel_size_in_meters = ee.Image.pixelArea().sqrt()
distance_meters_image = distance_pixels_image.multiply(pixel_size_in_meters)

Converting pixel distance to approximate METERS using pixelArea()...


In [ ]:
# --- 7. Mask distances greater than the threshold ---
print(f"Masking out all distances > {MAX_DISTANCE_METERS_THRESHOLD} meters...")
# The .lte() function means "less than or equal to".
# The mask keeps only the pixels that meet this condition.
final_distance_image = distance_meters_image.updateMask(
    distance_meters_image.lte(MAX_DISTANCE_METERS_THRESHOLD)
)

Masking out all distances > 1000 meters...


In [ ]:
# --- 8. Simplify the final image ---
print("Simplifying final image by dividing by 10, rounding, and converting to integer...")
# This results in an image with values from 0-100. toUint8 is efficient for this range.
simplified_distance_image = final_distance_image.divide(10).round().toUint8()


Simplifying final image by dividing by 10, rounding, and converting to integer...


In [ ]:
# --- 9. Clip Result ---
print("\nClipping final simplified image to AOI...")
final_simplified_image_clipped = simplified_distance_image.clip(aoi_geometry)


Clipping final simplified image to AOI...


In [ ]:
# --- 10. Visualize the final simplified Map ---
print("\nPreparing map visualization...")
Map = geemap.Map()
Map.centerObject(aoi_geometry, 4)

# --- UPDATED: Visualization for the new 0-100 range ---
dnn_vis_params = {
  'min': 0,
  'max': 100, # Corresponds to the original 1000m max distance
  'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000'] # Blue=Near -> Red=Far
}
print(f"Visualizing simplified distances from 0 to 100.")

# Add layers to the map
Map.addLayer(ee.Image().paint(aoi_geometry, 0, 2), {'palette': '000000'}, 'AOI Boundary')
Map.addLayer(soil_points, {'color': 'FF0000'}, 'Filtered Soil Samples (Red)')
Map.addLayer(
    final_simplified_image_clipped,
    dnn_vis_params,
    'Simplified Distance (0-100)'
)

print("\nScript finished. Displaying map.")
Map


Preparing map visualization...
Visualizing simplified distances from 0 to 100.

Script finished. Displaying map.


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…